# Custom Transformers

# Contenido
1. BaseEstimator y TransformerMixin de scikit-learn
  * 1.1 - Qué es scikit-learn?
  * 1.2 - Transformación de datos y transformadores personalizados
  * 1.3 - Estimator
  * 1.4 - Transformer
  * 1.5 - Transformers con BaseEstimator y TransformerMixin
2. Ejemplo: Obtener mean (promedio) y std (desviación estandar)
3. Ejercicio





# 1 - BaseEstimator y TransformerMixin de scikit-learn

## 1.1 Qué es scikit-learn?
Scikit-Learn es una librería gratuita para Python. Cuenta con **algoritmos de clasificación, regresión, clustering y reducción de dimensionalidad**. Además, presenta la compatibilidad con otras librerías de Python como NumPy, SciPy y matplotlib.


## 1.2 Transformación de datos y transformadores personalizados
Antes de entrenar modelos con algún algoritmo, es necesario preprocesar los datos. Esto es importante ya que permite "limpiar" la información, eliminar datos nulos, ajustar o simplemente agregar atributos extra.

A menudo necesitamos implementar alguna funcionalidad que no existe en scikit-learn o cualquier otro paquete, si nos ajustamos a la API de scikit-learn, podemos limitarnos a desarrollar un transformador / estimador personalizado y nuestro código interactuará muy bien con los módulos de scikit-learn. 

## 1.3 Estimator
Un estimador es un objeto que se ajusta a un modelo basado en algunos datos de entrenamiento y es capaz de inferir algunas propiedades en nuevos datos. Puede ser, por ejemplo, un clasificador o un regresor.

Todos los estimadores integrados también tienen un método `set_params`, que establece parámetros independientes de los datos (anulando los valores de parámetros anteriores pasados ​​a `__init__`).

Todos los estimadores en el código base principal de scikit-learn deben heredar de `sklearn.base.BaseEstimator`.

Los estimadores predicen un nuevo valor de salida `y` dado un valor de entrada `X`.



## 1.4 Transformer
Los transformadores son clases que permiten transformaciones de datos mientras procesan previamente los datos para el aprendizaje automático. 

Ejemplos de transformadores en Scikit-Learn son `SimpleImputer`, `MinMaxScaler`, `OrdinalEncoder`, `PowerTransformer`, por nombrar algunos.

Los transformadores implementan un método llamado `transform`. El caso de uso aplica para:

* `fit`, donde algunos parámetros se pueden aprender de `X` e `y`. En general, el método `fit` opera con los datos de entrenamiento y almacena algún estado resultante en el objeto.
* `transform`, donde `X` sera transformado usando los parametros aprendidos durante `fit`.

## 1.5 Transformers con clases `BaseEstimator` y `TransformerMixin`
Cuando se crea un transformador, es necesario crear una clase que herede de ambas bibliotecas `sklearn.base.BaseEstimator` y `sklearn.base.TransformerMixin`.

El resultado final va a permitir que un modelo no solo sea ajustado para predicciones, sino también usado en combinación con otras herramientas de scikit-learn como `grid search` o `pipelines`, que pueden ser empleados para hacer que nuestro código sea más limpio y más fácil de mantener.


### `BaseEstimator`
El estimador (estimator) es la pieza central de un transformer, regressor y classifier en `sklearn.base.BaseEstimator`. Todos los estimadores en scikit-learn se derivan de esta clase. 

`BaseEstimator` proporciona, entre otras cosas, una implementación predeterminada para los métodos `get_params` y `set_params`, que son útiles para poder aplicar GridSearchCV para el ajuste de parámetros de forma automática, y se comporte bien con otros cuando se combina en otros pipelines.

### `TransformerMixin`
Es  una clase base cuyos métodos `fit`y `transform` van a permitir realizar las transformaciones que nosotros les indiquemos.

### Uso

Todo lo que se necesita para hacer uso de estas clases es crear una clase e implementar tres métodos: `fit()` (retornando a sí mismo), `transform()` y `fit_transform()`. Puede obtener el último simplemente agregando `TransformerMixin` como clase base. Si agrega BaseEstimator como clase base (y evita *args y **kargs en su constructor), también obtendrá dos métodos adicionales (`get_params()` y `set_params()`) que serán útiles para el ajuste automático de hiperparámetros.

**Notas** 

La API de scikit-learn impone al método `fit` a solot retornar `self`, es decir, el mismo objeto. Esto permite que los metodos `fit` y `transform` sean impuestos por `sklearn.base.TransformerMixin`. Se hace esto debido a que a que se espera que el metodo `fit` tenga `X` y `y` como entradas (o argumentos). La funcion `transform`solo toma `X` como entrada y se espera que retorne una versión transformada de `X`.

# 2 - Ejemplo: Obtener mean (promedio) y std (desviación estándar)
En este ejemplo, el objetivo es generar un transformador personalizado que calcule el promedio de los datos y agregar dos atributos extra. 

* Crear un dataframe con datos usando Pandas para simular la información de entrada en formato crudo (raw), con las siguientes columnas: 
  * `edad`: numeros enteros, como 60, 87, 54
  * `estatura`: en metros, como 1.65, 1.80, 1.70
  * `peso`: numeros enteros, como 60, 87, 54
* Crear el transformador heredando la clase `BaseEstimator` y `TransformerMixin`.
* Usar el método `fit()` para calcular el promedio
* Usar el método `transform()` para agregar las siguientes columnas extra.
  * `imc`: que se refiere al indice de masa corporal, que se calcula con la fórmula $imc=peso / estatura^{2}$.
  * `promedio_x_imc`: multiplicación de el promedio por el `imc` calculado.
* Imprimir el resultad en como un dataframe de Pandas.

**Nota**

En este caso, vamos a usar Numpy para procesar los datos, pero perfectamente se puede hacer con Pandas. Solo que habra que convertir el arreglo de Numpy a un dataframe de Pandas.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
np.set_printoptions(suppress=True)

# Crear arreglo con edad, estatura, peso
my_array = [[20, 1.50, 55] ,[18, 1.70, 65] ,[23, 1.54, 60] ,[27, 1.60, 50],[25, 1.80, 90]]

# Conversion a pandas dataframe
df = pd.DataFrame(my_array, columns = ['edad','estatura', 'peso'])

# Column index
age_ix, height_ix, weight_ix = 0, 1, 2 # deberian ser constantes

class CustomTransformer(BaseEstimator, TransformerMixin):

  def __init__(self):
    pass

  def fit(self, X):
    self.mean = np.mean(X, axis=0) # axis=0 calcula las columnas, axis=1 las filas
    print(self.mean)
    return self

  def transform(self, X):
    imc = X[:, weight_ix] / (np.power(X[:, height_ix], 2))
    mean_imc = self.mean[0] * imc
    return np.c_[X, imc, mean_imc]


tr = CustomTransformer()
# tr.fit(df.values()) 
tr.fit(df.to_numpy()) # to_numpy()
people_extra_attributes = tr.transform(df.to_numpy())
print(people_extra_attributes)

[22.6    1.628 64.   ]
[[ 20.           1.5         55.          24.44444444 552.44444444]
 [ 18.           1.7         65.          22.49134948 508.30449827]
 [ 23.           1.54        60.          25.29937595 571.76589644]
 [ 27.           1.6         50.          19.53125    441.40625   ]
 [ 25.           1.8         90.          27.77777778 627.77777778]]


In [ ]:
df.head(5)

,edad,estatura,peso
0,20,1.50,55
1,18,1.70,65
2,23,1.54,60
3,27,1.60,50
4,25,1.80,90


Se convierte de Numpy array a Pandas dataframe, y añaden los nombres de las columnas.

In [ ]:
type(people_extra_attributes)

numpy.ndarray

In [ ]:
COL_NAMES = 'edad','estatura', 'peso', 'imc', 'promedio_edad_x_imc'
people_extra_attributes = pd.DataFrame(
    people_extra_attributes,
    columns=COL_NAMES)
people_extra_attributes.head() # Se imprimen los 5 datos

,edad,estatura,peso,imc,promedio_edad_x_imc
0,20.0,1.50,55.0,24.444444,552.444444
1,18.0,1.70,65.0,22.491349,508.304498
2,23.0,1.54,60.0,25.299376,571.765896
3,27.0,1.60,50.0,19.531250,441.406250
4,25.0,1.80,90.0,27.777778,627.777778


In [ ]:
type(people_extra_attributes)

pandas.core.frame.DataFrame

# 3 - Ejercicios

## Ejercicios de transformadores personalizados
Crear un transfomador personalizado que permita duplicar las columnas y agregar al final un "_nan" al nombre de ellas para identificar datos faltantes o nulos en cada fila.

* Cargar el dataset en formato CSV.
* Leer el dataframe con pandas.
* Crea tu transformador personalizado que solo identifique datos nulos en los campos numericos: `pclass`, `age`, `sibsp`, `parch`, `fare`
  
  * Entrada: columnas y filas con datos sin preprocesar.

age      | fare    
---------|---------
25       |    NaN  
NaN      |  232.2  
  
  * Salida: combinación de columnas de entrada con columnas que indiquen datos faltantes.

age      | fare    | age_nan | fare_nan
---------|---------|---------|-----
25       |    NaN  |    0    |   1
NaN      |  232.2  |    1    |   0

* Colocar el nombre de las columnas (transformador returna un arreglo Numpy, es decir, solo números).
* Imprimir los resultados.

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

NUMERICAL_VARS = ['pclass','age','sibsp','parch','fare']

# Crear custom transformer
class MissingIndicator(BaseEstimator, TransformerMixin):
    
    def __init__(self, variables):
        if not isinstance(variables, list):
            self.variables = [variables]
        else:
            self.variables = variables
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        X = X.copy()
        for var in self.variables:
            X[var+'_nan'] = X[var].isnull().astype(int)
        
        return X


# Leer el csv sin aplicar transformaciones
df = pd.read_csv("/content/raw-data.csv")

# Imprimir los primeros datos
print(df.head(10))

mi = MissingIndicator(variables=NUMERICAL_VARS)
# Aplicar las transformaciones
df_mi = mi.transform(df)

# Imprimir resultados despues de las transformaciones
df_mi.head(20)

   pclass  survived     sex      age  sibsp  parch      fare cabin embarked  \
0       1         1  female  29.0000      0      0  211.3375    B5        S   
1       1         1    male   0.9167      1      2  151.5500   C22        S   
2       1         0  female   2.0000      1      2  151.5500   C22        S   
3       1         0    male  30.0000      1      2  151.5500   C22        S   
4       1         0  female  25.0000      1      2  151.5500   C22        S   
5       1         1    male  48.0000      0      0   26.5500   E12        S   
6       1         1  female  63.0000      1      0   77.9583    D7        S   
7       1         0    male  39.0000      0      0    0.0000   A36        S   
8       1         1  female  53.0000      2      0   51.4792  C101        S   
9       1         0    male  71.0000      0      0   49.5042   NaN        C   

    title  
0    Miss  
1  Master  
2    Miss  
3      Mr  
4     Mrs  
5      Mr  
6    Miss  
7      Mr  
8     Mrs  
9      Mr 

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_nan,age_nan,sibsp_nan,parch_nan,fare_nan
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss,0,0,0,0,0
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master,0,0,0,0,0
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss,0,0,0,0,0
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr,0,0,0,0,0
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs,0,0,0,0,0
5,1,1,male,48.0000,0,0,26.5500,E12,S,Mr,0,0,0,0,0
6,1,1,female,63.0000,1,0,77.9583,D7,S,Miss,0,0,0,0,0
7,1,0,male,39.0000,0,0,0.0000,A36,S,Mr,0,0,0,0,0
8,1,1,female,53.0000,2,0,51.4792,C101,S,Mrs,0,0,0,0,0
9,1,0,male,71.0000,0,0,49.5042,NaN,C,Mr,0,0,0,0,0


**Nota**

Como la funcion `transform` no necesita calculos adicionales, solo retorna `self`. Aun asi, se necesita definir este metodo `fit` dado que scikit-learn espera que todos los estimadores tengan uno, incluso cuando no hagan nada.

# Fuentes
* https://sklearn-template.readthedocs.io/en/latest/user_guide.html
* https://stackoverflow.com/questions/66521182/explaination-for-a-code-snippet-related-to-classes-in-python
* https://scikit-learn.org/stable/developers/develop.html
* https://stackoverflow.com/questions/54899647/what-is-the-difference-between-transformer-and-estimator-in-sklearn


Ejemplos de CustomTransformers
* https://github.com/zachariaszblazej/AirBnb_NY_Price_Prediction/blob/ddef0ff7cabd30725700498aeb251c06c5a98eb1/Classes_And_Functions.py
* https://github.com/matthewgalloway/salary_predictions/blob/f04dfb6798324c3d1354c91b41ea9cc7d74a5641/preprocessors.py

